In [29]:
import pyspark.sql.functions as f
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf, SparkContext, HiveContext

In [30]:
executionDate = input("Input date you want transform data from HDFS DataLake and save to Hive Storage: ")

In [31]:
executionDate

'2020-05-12'

In [32]:
runTime = executionDate.split("-")
year = runTime[0]
month = runTime[1]
day = runTime[2]

In [33]:
# create spark session
spark = SparkSession \
   .builder \
   .appName("Daily Gross Revenue Report") \
   .config('hive.exec.dynamic.partition', 'true') \
   .config('hive.exec.dynamic.partition.mode', 'nonstrict') \
   .config('spark.sql.warehouse.dir', 'hdfs://localhost:9000/user/hive/warehouse') \
   .enableHiveSupport() \
   .getOrCreate()
#   

In [34]:
# load data to spark df
orders_df = spark.read.parquet('hdfs://localhost:9000/datalake/orders').drop("year", "month", "day")
order_detail_df = spark.read.parquet('hdfs://localhost:9000/datalake/order_detail').drop("year", "month", "day")
products_df = spark.read.parquet('hdfs://localhost:9000/datalake/products').drop("year", "month", "day", "created_at")
inventory_df = spark.read.parquet('hdfs://localhost:9000/datalake/inventory').drop("year", "month", "day")

In [35]:
orders_df.show(5)

+---+--------+----------+----------+
| id|quantity|created_at|product_id|
+---+--------+----------+----------+
|  1|       1|2009-01-25|    331449|
|  2|       1|2019-09-13|    182256|
|  3|       2|2004-05-04|    108399|
|  4|       3|2011-02-20|     81461|
|  5|       3|2007-07-11|    136274|
+---+--------+----------+----------+
only showing top 5 rows



In [36]:
order_detail_df.show(5)

+---+------+-----------+--------+-------+
| id| total|    payment|order_id|user_id|
+---+------+-----------+--------+-------+
|  1|710051|credit_card|       1| 209279|
|  2|375643|       cash|       2| 242546|
|  3|975362|       cash|       3| 135215|
|  4|417644|credit_card|       4| 111433|
|  5|481473|credit_card|       5|  44346|
+---+------+-----------+--------+-------+
only showing top 5 rows



In [37]:
products_df.show(5)

+---+--------+--------------+------------+------------+
| id|    make|         model|    category|inventory_id|
+---+--------+--------------+------------+------------+
|  1|     BMW|      5 Series|Sedan, Wagon|      999830|
|  2| Mercury| Grand Marquis|       Sedan|      988335|
|  3|   Honda|          CR-V|         SUV|      986788|
|  4|Cadillac|           XT5|         SUV|      986910|
|  5|  Nissan|Titan Crew Cab|      Pickup|      988637|
+---+--------+--------------+------------+------------+
only showing top 5 rows



In [38]:
inventory_df.show(5)

+---+--------+
| id|quantity|
+---+--------+
|  1|     355|
|  2|     492|
|  3|     269|
|  4|     394|
|  5|     239|
+---+--------+
only showing top 5 rows



In [39]:
pre_df = orders_df \
    .filter(orders_df["created_at"] == "2019-09-13") \
    .join(order_detail_df, orders_df["id"] == order_detail_df["order_id"], "inner") \
    .join(products_df, orders_df["product_id"] == products_df["id"], "inner") \
    .join(inventory_df.select(f.col("quantity").alias("inv_quantity"), f.col("id")), products_df["inventory_id"] == inventory_df["id"], "inner")

In [40]:
pre_df.show(5)

+-------+--------+----------+----------+-------+------+-----------+--------+-------+------+---------+--------------------+--------+------------+------------+------+
|     id|quantity|created_at|product_id|     id| total|    payment|order_id|user_id|    id|     make|               model|category|inventory_id|inv_quantity|    id|
+-------+--------+----------+----------+-------+------+-----------+--------+-------+------+---------+--------------------+--------+------------+------------+------+
|1288949|       1|2019-09-13|     31260|1288949|728643|credit_card| 1288949|  82492| 31260|      BMW|                X6 M|     SUV|      978802|         384|978802|
| 331439|       3|2019-09-13|    283893| 331439|858674| instalment|  331439| 136994|283893|    Acura|                 TSX|   Sedan|      997702|         255|997702|
| 462327|       1|2019-09-13|    196266| 462327|974749|credit_card|  462327| 269750|196266|    MAZDA|B-Series Regular Cab|  Pickup|      981583|         371|981583|
|1601604| 

In [41]:
map_df = pre_df.groupBy("Make", "Model", "Category", "product_id", "inv_quantity") \
    .agg(
        f.sum("quantity").alias("Sales"),
        f.sum("total").alias("Revenue")
    )

In [42]:
result_df = map_df \
    .withColumn("LetfOver", f.col("inv_quantity") - f.col("Sales")) \
    .withColumn("year", f.lit(year)) \
    .withColumn("month", f.lit(month)) \
    .withColumn("day", f.lit(day)) \
    .select("Make", "Model", "Category", "Sales", "Revenue", "year", "month", "day", "LetfOver")

In [43]:
# df = spark.sql("show databases")

In [44]:
# df.show()

In [45]:
# spark.sql("create database test")
# df1 = spark.sql("show databases")

In [46]:
# df1.show()

In [47]:
# df = spark.sql("describe database reports")
# df.select('info_value').collect()

In [48]:
# spark.sql("create database reports")
# df2 = spark.sql("describe database reports")
# df2.select('info_value').collect()
# spark.sql("show databases").show()

In [49]:
result_df.write \
    .format("hive") \
    .partitionBy("year", "month", "day") \
    .mode("append") \
    .saveAsTable("reports.daily_gross_revenue")

23/08/03 01:25:14 INFO HiveMetaStore: 0: get_table : db=reports tbl=daily_gross_revenue
23/08/03 01:25:14 INFO audit: ugi=hadoop	ip=unknown-ip-addr	cmd=get_table : db=reports tbl=daily_gross_revenue	
23/08/03 01:25:14 INFO HiveMetaStore: 0: get_table : db=reports tbl=daily_gross_revenue
23/08/03 01:25:14 INFO audit: ugi=hadoop	ip=unknown-ip-addr	cmd=get_table : db=reports tbl=daily_gross_revenue	
23/08/03 01:25:14 INFO HiveMetaStore: 0: get_database: reports
23/08/03 01:25:14 INFO audit: ugi=hadoop	ip=unknown-ip-addr	cmd=get_database: reports	
23/08/03 01:25:14 INFO HiveMetaStore: 0: get_table : db=reports tbl=daily_gross_revenue
23/08/03 01:25:14 INFO audit: ugi=hadoop	ip=unknown-ip-addr	cmd=get_table : db=reports tbl=daily_gross_revenue	
23/08/03 01:25:14 INFO HiveMetaStore: 0: get_table : db=reports tbl=daily_gross_revenue
23/08/03 01:25:14 INFO audit: ugi=hadoop	ip=unknown-ip-addr	cmd=get_table : db=reports tbl=daily_gross_revenue	
23/08/03 01:25:15 INFO HiveMetaStore: 0: get_table

23/08/03 01:25:44 INFO HiveMetaStore: 0: get_table : db=reports tbl=daily_gross_revenue
23/08/03 01:25:44 INFO audit: ugi=hadoop	ip=unknown-ip-addr	cmd=get_table : db=reports tbl=daily_gross_revenue	
23/08/03 01:25:44 INFO HiveMetaStore: 0: get_table : db=reports tbl=daily_gross_revenue
23/08/03 01:25:44 INFO audit: ugi=hadoop	ip=unknown-ip-addr	cmd=get_table : db=reports tbl=daily_gross_revenue	
23/08/03 01:25:45 INFO HiveMetaStore: 0: get_table : db=reports tbl=daily_gross_revenue
23/08/03 01:25:45 INFO audit: ugi=hadoop	ip=unknown-ip-addr	cmd=get_table : db=reports tbl=daily_gross_revenue	
23/08/03 01:25:45 INFO HiveMetaStore: 0: get_table : db=reports tbl=daily_gross_revenue
23/08/03 01:25:45 INFO audit: ugi=hadoop	ip=unknown-ip-addr	cmd=get_table : db=reports tbl=daily_gross_revenue	
23/08/03 01:25:45 INFO Hive: New loading path = hdfs://localhost:9000/user/hive/warehouse/reports.db/daily_gross_revenue/.hive-staging_hive_2023-08-03_01-25-15_104_2323326712391549837-1/-ext-10000/yea